In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tiktoken
import numpy as np
from Single_Block_Fortnite import EmbeddingNN, AttentionNN, NormNN, FFN


# For JSONL




In [26]:
import json
import torch
from transformers import AutoTokenizer

# 1. Load dataset
import json

dataset = []
with open("C:/Users/PC/Desktop/Important Data/Rem_Data.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:  # skip blank lines
            continue
        try:
            dataset.append(json.loads(line))
        except json.JSONDecodeError as e:
            print("Skipping bad line:", line)
            print("Error:", e)

print("Sample entry:", dataset[0])  # sanity check

# 2. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# 3. Collect all texts
all_texts = [entry["text"] for entry in dataset if "text" in entry]


# 4. Encode all texts into one long sequence
ids = []
for t in all_texts:
    encoded = tokenizer.encode(t, add_special_tokens=False)
    encoded.append(tokenizer.eos_token_id or 50256)  # add EOS after each line
    ids.extend(encoded)

# 5. Convert to tensor
device = "cuda" if torch.cuda.is_available() else "cpu"
data = torch.tensor(ids, dtype=torch.long, device=device)

print("Total tokens:", len(ids))
print("Tensor shape:", data.shape)
vocab_size = tokenizer.vocab_size

Sample entry: {'scene_id': 'rem_if_batch1_001', 'speaker': 'rem', 'text': '……Thank you very much for your concern. But…', 'tone': 'polite', 'emotion': 'guarded', 'subtext': 'Acknowledges kindness but resists being spoiled'}
Total tokens: 15902
Tensor shape: torch.Size([15902])


In [27]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#text = ds
#with open("C:/Users/PC/Desktop/Important Data/Shakespeare_Data.txt", "r", encoding="utf-8") as f:
   # text = f.read()
  
    
#integers = tokenizer.encode(str(text)) 
#vocab_size = tokenizer.n_vocab  # Total number of tokens in the tokenizer
#end_token_id = 50256  
#integers.append(end_token_id) 
#data = torch.tensor(integers , dtype=torch.long).to(device)


#print(vocab_size)
#data = torch.tensor(integers , dtype = torch.long)
#batch_x = data[i:i+seq_len].unsqueeze(0)
   #batch_x = batch_x.repeat(batch_size, 1)

In [28]:
class TransformerBlock(nn.Module):
    def __init__(self, hidden_dim, num_heads):
        super().__init__()

        self.attention = AttentionNN(hidden_dim, num_heads)
        self.ffn = FFN(hidden_dim)
        self.norm1 = NormNN(hidden_dim)
        self.norm2 = NormNN(hidden_dim)

    def forward(self, X):
        attention_output = self.attention(X)
        X = X + attention_output

        pog_X = self.norm1(X)

        pogger_X = self.ffn(pog_X) + pog_X

        poggers_X = self.norm2(pogger_X)

        return poggers_X

In [29]:
class Transformer(nn.Module):
    def __init__(self,vocab_size , hidden_dim , num_heads,max_sequence_length, N):
        super().__init__()

        self.embedding = EmbeddingNN(vocab_size, hidden_dim, max_sequence_length)
        self.blocks = nn.ModuleList([TransformerBlock(hidden_dim, num_heads) for i in range(N)])
        self.linear = nn.Linear(hidden_dim,vocab_size)

    def forward(self, X):
        X = self.embedding(X)
        for block in self.blocks:
            X = block(X)
        X = self.linear(X)

        return X

In [30]:
#tokenizer = tiktoken.get_encoding("gpt2")

#integers = tokenizer.encode(text)
#decode_dict = {i : chars for i , chars in enumerate(chars)}
#encode_dict = {chars : i for i , chars in enumerate(chars)}

#encode = lambda text : [encode_dict[i] for i in text]
#decode = lambda indices : ''.join([decode_dict[i] for i in indices])

#encoded_text = encode(text)

#print(encode_dict['<END>'])
#print("Max index in data:", max(encoded_text), "Vocab size:", vocab_size)
 # usually 50256 for GPT-2



In [31]:
batch_size =8
hidden_dim = 768
max_sequence_length = 512
num_heads = 8
seq_len = 128
epochs = 250

In [32]:
Transformer_model = Transformer(vocab_size, hidden_dim, num_heads, max_sequence_length, 12).to(device)
optimizer = optim.Adam(Transformer_model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
n = 0
loss = None
batch_y = []
for epoch in range(epochs):
    n = n+1
    print(f"Epoch Number: {n}")
    for start in range(0, len(data) - batch_size*seq_len, batch_size*seq_len):
    
        end = start+batch_size*seq_len
        if end + 1 > len(data):
            break
    
        batch_x = data[start : end].view(batch_size, seq_len).to(device)
        batch_y = data[start+1 : end+1].view(batch_size, seq_len).to(device)
        
        logits = Transformer_model(batch_x).to(device)
    
        logits = logits.view(-1 , vocab_size)
        batch_y = batch_y.view(-1)
    
        loss = criterion(logits, batch_y)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
if loss is not None:
    print("Final loss:", loss.item())
else:
    print("No batches were processed. Loss not defined.")

Epoch Number: 1
Epoch Number: 2
Epoch Number: 3
Epoch Number: 4
Epoch Number: 5
Epoch Number: 6
Epoch Number: 7
Epoch Number: 8
Epoch Number: 9
Epoch Number: 10
Epoch Number: 11
Epoch Number: 12
Epoch Number: 13
Epoch Number: 14
Epoch Number: 15
Epoch Number: 16
Epoch Number: 17
Epoch Number: 18
Epoch Number: 19
Epoch Number: 20
Epoch Number: 21
Epoch Number: 22
Epoch Number: 23
Epoch Number: 24
Epoch Number: 25
Epoch Number: 26
Epoch Number: 27
Epoch Number: 28
Epoch Number: 29
Epoch Number: 30
Epoch Number: 31
Epoch Number: 32
Epoch Number: 33
Epoch Number: 34
Epoch Number: 35
Epoch Number: 36
Epoch Number: 37
Epoch Number: 38
Epoch Number: 39
Epoch Number: 40
Epoch Number: 41
Epoch Number: 42
Epoch Number: 43
Epoch Number: 44
Epoch Number: 45
Epoch Number: 46
Epoch Number: 47
Epoch Number: 48
Epoch Number: 49
Epoch Number: 50
Epoch Number: 51
Epoch Number: 52
Epoch Number: 53
Epoch Number: 54
Epoch Number: 55
Epoch Number: 56
Epoch Number: 57
Epoch Number: 58
Epoch Number: 59
Epoch 

In [33]:
torch.save(Transformer_model.state_dict(), "Sigma_Transformer.pth")

In [40]:
text = input()
input_ids = tokenizer.encode(text)
input_text = torch.tensor([input_ids]).to(device)

inference = []
temperature = 0.8
while True:
    
    logits = Transformer_model(input_text).to(device)
    next_token = logits[:,-1 ,:]
    probs = torch.softmax(next_token / temperature, dim=-1)
    next_token_id = torch.multinomial(probs, 1)
    input_text = torch.cat([input_text, next_token_id], dim=1)

    inference.append(next_token_id.item())

    if next_token_id.item() == 50256 or len(inference) >=  200:
        break
        
print(tokenizer.decode(inference))


 hello rem


 fulfilled make Tia-sama of us at between stories told are giving birth to our child….. are important.<|endoftext|>
